# 감성 분석

In [1]:
import re
import pandas as pd
from konlpy.tag import Okt

## 1. 데이터 로드

In [2]:
## 감성 사전 ##
keywords = pd.read_excel('data/words.xlsx')

positive = list(keywords['positive'][keywords['positive'].notnull()])
negative = list(keywords['negative'][keywords['negative'].notnull()])

positive = list(set(positive))
negative = list(set(negative))

# 불용어
stopwords = ['의','가','이','은','들','는','좀','걍','과','도','를','으로','자','에','와','한','하다', '이다', '에서']

In [3]:
## 뉴스 데이터 로드 ##

# 크롤링한 뉴스 데이터의 시작, 끝 날짜
start_date = '2023.06.01'
end_date = '2023.06.15'

df = pd.read_excel('data/stock_news_result_{}.xlsx'.format(start_date+'-'+end_date), index_col = 'Unnamed: 0')

## 2. 전처리

In [4]:
## 전처리 함수 ##

regex = re.compile('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 ]')

def Pre_processing(df, stopwords):

    # 중복 행 제거
    df.drop_duplicates(['title'], inplace = True)
    df = df.reset_index(drop = True)
    
    # 정규표현식 -> 한글만 남기기
    f = lambda x: re.sub(regex, "", x)
    df['title'] = df['title'].map(f)
        
    # 토큰화
    okt = Okt()
    tokenized = []

    for i in df['title']:
        temp_x = []
        temp_x = okt.morphs(i, stem = False)
        temp_x = [word for word in temp_x if not word in stopwords] 
        tokenized.append(temp_x)
    
    return df, tokenized

In [5]:
# 전처리 수행
df_pcd, token = Pre_processing(df, stopwords)

# 토큰화된 문자열 새로운 칼럼에 넣기
df_pcd['title_tokenized'] = token

df_pcd

,stock,press,title,title_tokenized
0,삼성전자,파이낸셜뉴스,나흘간 조억 팔아치운 외국인 삼성전자는 더 담자,"[나흘, 간, 조억, 팔아치운, 외국인, 삼, 성, 전자, 더, 담자]"
1,삼성전자,한국경제,송봉섭 삼성전자서비스 대표능률협회컨설팅 특별 공헌상,"[송봉섭, 삼성, 전자, 서비스, 대표, 능률, 협회, 컨설팅, 특별, 공헌, 상]"
2,삼성전자,한겨레,세계 파운드리 업계 삼성전자인텔 파전 재편 본격화,"[세계, 파운드리, 업계, 삼성, 전자, 인텔, 파전, 재편, 본격, 화]"
3,삼성전자,한국경제,삼성전자자이대한항공하이카신한 년째 사랑받는 위,"[삼, 성, 전자, 대한항공, 하, 카신, 년, 째, 사랑받는, 위]"
4,삼성전자,전자신문언론사 선정,삼성전자 텍사스에 네트워크혁신센터 구축,"[삼성, 전자, 텍사스, 네트워크, 혁신, 센터, 구축]"
...,...,...,...,...
1900,넷마블,디지털타임스,넷마블문화재단 일까지 찾아가는 게임소통학교 참가자 모집,"[넷마블, 문화재단, 일, 까지, 찾아가는, 게임, 소통, 학교, 참가자, 모집]"
1901,넷마블,디지털타임스,상반기 히트상품 웹툰 성공공식 게임에 적용,"[상반기, 히트, 상품, 웹툰, 성공, 공식, 게임, 적용]"
1902,넷마블,서울신문,의 거짓세븐나이츠 황금알 거위 개척에 승부수 던졌다,"[거짓, 세븐, 나이, 츠, 황금알, 거위, 개척, 승부, 수, 던졌다]"
1903,넷마블,아시아경제,비전프로 등장에 불붙는 시장국내 게임사 바빠진다,"[비전, 프로, 등장, 불, 붙는, 시장, 국내, 게임, 사, 바빠진다]"


## 3. 감성분석 및 스코어링(scoring)

In [6]:
# 클래스 정의
class NewsScore():
    def make(self, name):
        # 종목명
        self.name = name
        # 해당 종목의 기사
        self.news = pd.DataFrame() 
        # 스코어링 된 기사
        self.news_final = pd.DataFrame()
        # 해당 종목의 최종 스코어 
        self.score = 0 

In [7]:
# 뉴스 데이터가 있는 종목들로 최종 종목 리스트 정의
stocks_final = list(df_pcd['stock'].unique())

In [8]:
# 최종 종목별 인스턴스 생성
news_score = [NewsScore() for i in range(len(stocks_final))] 

In [9]:
for idx_s, s in enumerate(stocks_final):
    # 인스턴스에 종목명 넣기
    news_score[idx_s].make(s)
    # 해당 종목의 뉴스 df 넣기
    news_score[idx_s].news = df_pcd[df_pcd['stock'] == s] 
    news_score[idx_s].news.reset_index(drop = True, inplace = True)
    
    for t in range(len(news_score[idx_s].news)):
        # 토큰화된 제목 가져오기
        target_title = news_score[idx_s].news.loc[t, 'title_tokenized'] 
        
        for w in target_title:
            # 긍정 또는 부정 단어에 해당할 경우
            if w in positive or w in negative: 
                if w in positive:
                    # 긍정 단어일 경우 +1점
                    news_score[idx_s].score += 1 

                else:
                    # 부정 단어일 경우 -1점
                    news_score[idx_s].score -= 1 
                # 스코어링된 기사 넣기
                news_score[idx_s].news_final = pd.concat([news_score[idx_s].news_final, news_score[idx_s].news.loc[[t], :]]) 

In [10]:
# 스코어 데이터프레임 정의
df_score = pd.DataFrame({
        'stock':news_score[i].name,
        'score':news_score[i].score} for i in range(len(news_score)))

df_score.sort_values('score', ascending = False, inplace = True)
df_score.reset_index(drop = True, inplace = True)

df_score

,stock,score
0,삼성전자,37
1,SK,20
2,카카오,19
3,LG,15
4,LG에너지솔루션,10
5,현대차,4
6,KT&G,4
7,삼성물산,4
8,기업은행,4
9,KB금융,3


## 4. 결과 저장

In [11]:
# 저장할 스코어 상위 종목 개수
best_stocks_number = 5
# 저장할 스코어 하위 종목 개수
worst_stocks_number = 5 

best_stocks_list = list(df_score.loc[:best_stocks_number-1, 'stock'])
worst_stocks_list = list(df_score.loc[df_score.index[int('-'+str(worst_stocks_number))]:, 'stock'])

In [12]:
df_news_best = pd.DataFrame()
df_news_worst = pd.DataFrame()

# 스코어 상위 종목들에 해당하는 뉴스만 골라 넣기
for s in best_stocks_list:
    for i in range(len(news_score)):
        if s == news_score[i].name:
            df_news_best = pd.concat([df_news_best, news_score[i].news_final])

# 스코어 하위 종목들에 해당하는 뉴스만 골라 넣기         
for s in worst_stocks_list:
    for i in range(len(news_score)):
        if s == news_score[i].name:
            df_news_worst = pd.concat([df_news_worst, news_score[i].news_final])

In [13]:
 # 불필요한 칼럼 제거
df_news_best.drop('title_tokenized', axis = 1, inplace = True)
df_news_worst.drop('title_tokenized', axis = 1, inplace = True)

In [14]:
# 스코어 상위 종목 및 뉴스
df_news_best.to_excel('results/best_stocks_{}.xlsx'.format(start_date+'-'+end_date))

# 스코어 하위 종목 및 뉴스
df_news_worst.to_excel('results/worst_stocks_{}.xlsx'.format(start_date+'-'+end_date))

# 종목별 스코어
df_score.to_excel('results/score_{}.xlsx'.format(start_date+'-'+end_date))